In [1]:
import pyautogui
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import bs4
import pandas as pd
import time
import re
import os

path_text = pyautogui.prompt(text='โฟลเดอร์ที่ไฟล์อยู่',title='กรอกค่าที่ต้องการ แล้วกด OK')
chromedriver_path = os.path.join(path_text,'chromedriver')
website = 'https://www.shopee.co.th'

left_tag_list = []
right_tag_list = []
title_list = []
all_coupon_list = []
price_list = []
sales_list = []
all_stars_list = []
all_address = []
all_shop_link = []
advertise_list = []
shop_name_list = []

def open_shopee():
    driver = webdriver.Chrome(executable_path=chromedriver_path)
    driver.maximize_window()
    driver.get(website)
    return driver

def close_popup(driver):
    while True:
        try:
            thai_button = driver.find_element_by_class_name('shopee-button-outline')
            thai_button.click()
            break
        except:
            pass
    time.sleep(1)
    close_button = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
    close_button.click()
    return driver

def search_keyword(driver,keyword):
    search_bar = driver.find_element_by_class_name('shopee-searchbar-input__input')
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.ENTER)
    return driver

def get_advertisement_data(product):
    try:
        advertise = product.find('div',{'class':'STY6D1'}).text
    except:
        advertise = 'N'
    advertise_list.append(advertise)

def get_left_tag_data(product):
    left_tag = product.find('div',{'class':'_39tdMd'})
    if left_tag:
        left_tag_text = left_tag.text
        if left_tag_text:
            left_tag_list.append('recommend')
        else:
            left_tag_list.append('mall')
    else:
        left_tag_list.append('normal')

def get_right_tag_data(product):
    right_tag = product.find('div',{'class':'_2qsK5P'})
    if right_tag:
        right_tag_text = right_tag.text
        right_tag_list.append(right_tag_text)
    else:
        right_tag_list.append(0)

def get_title_data(product):
    title = product.find('div',{'class':'_10Wbs- _2STCsK _3IqNCf'})
    title_list.append(title.text)


def get_coupon_data(product):
    coupon = product.find('div',{'class':'_11xQ9c'})
    if coupon:
        coupon_list = []
        all_coupon = coupon.findChildren(recursive=False)
        for c in all_coupon:
            coupon_list.append(c.text)
        all_coupon_list.append(coupon_list)
    else:
        all_coupon_list.append([])

def get_price_data(product):
    price = product.find('div',{'class':'_3_FVSo'})
    price_list.append(price.text)

    sales = product.find('div',{'class':'_1uq9fs'})
    sales_text = sales.text
    if sales_text:
        sales_list.append(sales_text)
    else:
        sales_list.append(0)

def get_stars_data(product):
    stars = product.find('div',{'class':'_3Ho4WH'})
    all_stars = stars.find_all('div',{'class':'shopee-rating-stars__star-wrapper'})
    stars_list = []
    for s in all_stars:
        stars_list.append(s.find('div',{'class':'shopee-rating-stars__lit'}).get('style'))
    all_stars_list.append(stars_list)

def get_address_data(product):
    address = product.find('div',{'class':'_1Ahoby'})
    all_address.append(address.text)

def get_data(driver):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    all_product = soup.find_all('div',{'class':'col-xs-2-4 shopee-search-item-result__item'})

    for product in all_product:
        get_advertisement_data(product)
        get_left_tag_data(product)
        get_right_tag_data(product)
        get_title_data(product)
        get_coupon_data(product)
        get_price_data(product)
        get_stars_data(product)
        get_address_data(product)
        all_shop_link.append('https://shopee.co.th'+product.find('a').get('href'))

def get_shop_name_data(all_shop_link,driver):
    for shop in all_shop_link:
        driver.get(shop)
        driver.execute_script("document.body.style.zoom='50%'")
        condition = True
        while condition:
            try:
                shop_name = driver.find_element_by_class_name('_1wVLAc').text
                condition = False
            except:
                pass
        print(shop_name)
        shop_name_list.append(shop_name)

def get_right_tag(text):
    if text == 0:
        return 0
    elif text:
        pattern = r'[0-9]+'
        return int(re.findall(pattern,text)[0]) / 100
    else:
        return 0

def right_tag_merge(report):
    right_tag_ser = report['right_tag'].apply(get_right_tag)
    right_tag_ser.name = 'right_tag_number'
    report = report.merge(right_tag_ser,left_index=True,right_index=True)
    return report


def coupon_merge(report):
    coupon_df = pd.DataFrame(list(report['coupon']))
    coupon_df.columns = [f'coupon_{num}' for num in range(len(coupon_df.columns))]
    
    coupon_baht_df = coupon_df.applymap(get_coupon_baht)
    coupon_baht_df.columns = [f'coupon_baht_{num}' for num in range(len(coupon_baht_df.columns))]
    report = report.merge(coupon_baht_df,left_index=True,right_index=True)
    
    coupon_percent_df = coupon_df.applymap(get_coupon_percent)
    coupon_percent_df.columns = [f'coupon_percent_{num}' for num in range(len(coupon_percent_df.columns))]
    report = report.merge(coupon_percent_df,left_index=True,right_index=True)
    
    return report


def get_coupon_baht(text):
    if text:
        pattern = r'[0-9]+'
        if '฿' in text:
            return int(re.findall(pattern,text)[0])

def get_coupon_percent(text):
    if text:
        pattern = r'[0-9]+'
        if '%' in text:
            return int(re.findall(pattern,text)[0]) / 100

def get_price(text):
    return int(text.split('฿')[-1].replace(',',''))

def price_merge(report):
    price_df = report['price'].apply(get_price)
    price_df.name = 'price_number'
    report = report.merge(price_df,left_index=True,right_index=True)
    return report

def get_sales(text):
    if text:
        pattern = r'[0-9\.]+'
        if 'พัน' in text:
            return float(re.findall(pattern,text)[0]) * 1000
        else:
            return int(re.findall(pattern,text)[0])

def sales_merge(report):
    sales_ser = report['sales'].apply(get_sales)
    sales_ser.name = 'sales_number'
    report = report.merge(sales_ser,left_index=True,right_index=True)
    return report

def get_star(data_list):
    pattern = r'[0-9]+'
    star_count = 0
    for star in data_list:
        star_count += (float(re.findall(pattern,star)[0]) / 100) * 1
    if star_count == 0:
        return None
    return star_count

def star_merge(report):
    star_df = report['stars'].apply(get_star)
    star_df.name = 'star_number'
    report = report.merge(star_df,left_index=True,right_index=True)
    return report

def main():
    keyword = pyautogui.prompt(text='Keyword ที่ต้องการ search',title='กรอกค่าที่ต้องการ แล้วกด OK')

    driver = open_shopee()
    close_popup(driver)
    search_keyword(driver,keyword)
    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(2)

    get_data(driver)
    get_shop_name_data(all_shop_link,driver)
    
    report = pd.DataFrame([title_list,left_tag_list,right_tag_list,all_coupon_list,price_list,sales_list,all_stars_list,all_address,all_shop_link,shop_name_list,advertise_list])
    report = report.transpose()
    report.columns = ['title','listing_type','right_tag','coupon','price','sales','stars','origin','shop_link','shop_name','advertisement']

    report = right_tag_merge(report)
    report = coupon_merge(report)
    report = price_merge(report)
    report = sales_merge(report)
    report = star_merge(report)
    report = report.drop(labels=['right_tag','coupon','price','sales','stars'],axis=1)
    report.to_excel(os.path.join(path_text,f'{keyword} on Shopee.xlsx'))
    driver.close()
    
main()

krapaoshop9
ur_story.th
ai.center
beyondgadget
logi.shop
youpin_thailand_store
gladful
huskshop
beyondgadget
mijiaofficial
micollectin
pngshopth
logi.shop
fantechthailand
hardware_corner
zeerwireless
rwc1dive1z
lg_flagshipstore
loveyman
ega_officialstore
seoulyoustore
newthinklife.store
suphot_asia
rz04prtuvz
aksolutionshop
pchanakarn.
logathailand
ur_story.th
gms_corporation
beyondgadget
razer.official.store
logathailand
gadgetvilla
zeerwireless
fuse.in.th
monitor2home
chichangcomputer
bananaonlineshop
gadgetvilla
lg_flagshipstore
pchanakarn.
giga_it
smartsunc
aukeythailandofficialstore
eexpresscorp
ok2home_official
logi.shop
razer.official.store
fuse.in.th
bananaonlineshop
mcwinner
yaiyai77
gadgetvilla
anitech_officialstore
fuse.in.th
gladful
ai.center
hiliboom2021
ecwmart
loeve007
